In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

import os
from nltk.tokenize import RegexpTokenizer
from hindi_tokenizer import Tokenizer
import re


from keras.layers import LSTM,Dense,Dropout,Input,Embedding,Bidirectional
from keras.models import Model

Using TensorFlow backend.
/home/suchith/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
data=pd.read_csv("hin.txt",delimiter='\t',header=None,quoting=3)
data.rename(columns={0:'english',1:'hindi'},inplace=True)
data

,english,hindi
0,Wow!,वाह!
1,Help!,बचाओ!
2,Jump.,उछलो.
3,Jump.,कूदो.
4,Jump.,छलांग.
5,Hello!,नमस्ते।
6,Hello!,नमस्कार।
7,Cheers!,वाह-वाह!
8,Cheers!,चियर्स!
9,Got it?,समझे कि नहीं?


In [3]:
s=data.iloc[2839,0]
s

"Don't you think the air conditioner is turned up too high in here?"

In [4]:
k=re.sub(r'\d+',r'',s)
k
k=k.replace(u',','')
k=k.replace(u'"','')
k=k.replace(u'(','')
k=k.replace(u')','')
k=k.replace(u':','')
k=k.replace(u'?','')
k=k.replace(u'!','')
k=k.replace(u';','')
k=k.replace(u'.','')
k

"Don't you think the air conditioner is turned up too high in here"

In [5]:

# doing something crazy
m=data.iloc[23,1]
print(m)
m=m[len(m)-1]


def remove_puntuation_english(s):
    k=re.sub(r'\d+',r'',s)
    k=k.replace(u',','')
    k=k.replace(u'"','')
    k=k.replace(u'(','')
    k=k.replace(u')','')
    k=k.replace(u':','')
    k=k.replace(u'?','')
    k=k.replace(u'!','')
    k=k.replace(u';','')
    k=k.replace(u'.','')
    
    return k

def remove_puntuation_hindi(s):
    k=re.sub(r'\d+',r'',s)
    k=k.replace(u',','')
    k=k.replace(u'"','')
    k=k.replace(u'(','')
    k=k.replace(u')','')
    k=k.replace(u':','')
    k=k.replace(u'.','')
    k=k.replace(m,'')
    k=k.replace(u'!','')
    k=k.replace(u';','')
    k=k.replace(u'?','')
    
    return k

data['english']=data['english'].apply(remove_puntuation_english)
data['hindi']=data['hindi'].apply(remove_puntuation_hindi)

data

मैं भूल गया।


,english,hindi
0,Wow,वाह
1,Help,बचाओ
2,Jump,उछलो
3,Jump,कूदो
4,Jump,छलांग
5,Hello,नमस्ते
6,Hello,नमस्कार
7,Cheers,वाह-वाह
8,Cheers,चियर्स
9,Got it,समझे कि नहीं


In [6]:
def func(s):
    return "_".join(s.split())

data['hindi']=data['hindi'].apply(func)
data['english']=data['english'].apply(func)

data

,english,hindi
0,Wow,वाह
1,Help,बचाओ
2,Jump,उछलो
3,Jump,कूदो
4,Jump,छलांग
5,Hello,नमस्ते
6,Hello,नमस्कार
7,Cheers,वाह-वाह
8,Cheers,चियर्स
9,Got_it,समझे_कि_नहीं


In [7]:
# i represent '\t' for start of the sentence and '\n' for end of the sentence
# i will add these only to suggestion column as they are target
def add(s):
    return '\t_'+s+'_\n'

data['hindi']=data['hindi'].apply(add)
data.head()

,english,hindi
0,Wow,\t_वाह_\n
1,Help,\t_बचाओ_\n
2,Jump,\t_उछलो_\n
3,Jump,\t_कूदो_\n
4,Jump,\t_छलांग_\n


In [8]:
# finding the maximum query length and maximum suggestion length
def spl(x):
    return len(x.split('_'))
max_length_query=data['english'].apply(spl).max()
max_length_suggestion=data['hindi'].apply(spl).max()
print("The maximum length of input is",max_length_query)
print("The maximum lenght of output is",max_length_suggestion)

The maximum length of input is 22
The maximum lenght of output is 27


In [9]:
input_words=set()
output_words=set()
for i in data['english']:
    for j in i.split('_'):
        input_words.add(j)

for i in data['hindi']:
    for j in i.split('_'):
        output_words.add(j)
        
# i am sorting just because it looks beautiful
input_words=sorted(list(input_words))
output_words=sorted(list(output_words))

num_encoder_tokens=len(input_words)
num_decoder_tokens=len(output_words)

print("number of unique input tokens",num_encoder_tokens)
print("number of unique output tokens",num_decoder_tokens)


# actually mapping between word and it's index

input_token=dict((word,i) for i,word in enumerate(input_words))
output_token=dict((word,i) for i,word in enumerate(output_words))

number of unique input tokens 2601
number of unique output tokens 2868


In [10]:

# setting number of examples as data.shape[0]
num_examples=data.shape[0]

# making numpy arrays
encoder_input_data=np.zeros((num_examples,max_length_query,num_encoder_tokens),dtype='float32')
decoder_input_data=np.zeros((num_examples,max_length_suggestion,num_decoder_tokens),dtype='float32')
decoder_output_data=np.zeros((num_examples,max_length_suggestion,num_decoder_tokens),dtype='float32')

# for query
for i in range(num_examples):
    # 0 column represents query 
    sen=data.iloc[i,0].split('_')
    for j,word in enumerate(sen):
        k=input_token[word]
        encoder_input_data[i,j,k]=1
        
# for suggestion
for i in range(num_examples):
    # 1 column represents suggestion
    sen=data.iloc[i,1].split('_')
    for j,word in enumerate(sen):
        k=output_token[word]
#         print(i,j,k,word)
        decoder_input_data[i,j,k]=1
        if j>0:
            decoder_output_data[i,j-1,k]=1
            
# for our sake we will print the shapes of the array
print("Encoder input data shape",encoder_input_data.shape)
print("Decoder input data shape",decoder_input_data.shape)
print("Decoder output data shape",decoder_output_data.shape)

Encoder input data shape (2867, 22, 2601)
Decoder input data shape (2867, 27, 2868)
Decoder output data shape (2867, 27, 2868)


In [11]:
latent_dim=300

def init():
    encoder_inputs=Input(shape=(None,))
    encoder_inputs=Embedding(num_encoder_tokens,latent_dim)(encoder_inputs)
    _,state_h,state_c,state_bh,state_bc=Bidirectional(LSTM(latent_dim,activation='tanh',return_state=True))(encoder_inputs)

    encoder_states=[state_h,state_c,state_bh,state_bc]

    decoder_inputs=Input(shape=(None,))
    decoder_inputs=Embedding(num_decoder_tokens,latent_dim)(decoder_inputs)

    decoder_outputs,_,_,_,_=Bidirectional(LSTM(latent_dim,return_state=True,return_sequences=True))(decoder_inputs,initial_state=encoder_states)
    decoder_outputs=Dense(num_decoder_tokens,activation='softmax')(decoder_outputs)

    print(decoder_inputs)
    
    model=Model(inputs=[encoder_inputs,decoder_inputs],outputs=decoder_outputs)
    
    return model

    
model=init()

model.compile(optimizer='adam',loss='categorical_cross_entropy')

model.summary()

AttributeError: 'list' object has no attribute 'get_shape'